### Классификация текстов

Используя ноутбук занятия (также размещен в папке _Materials_) и данные _fakenews_, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на _sklearn_ и выше 0.52 для методов на _pytorch_.

In [92]:
import torch
import pandas as pd

from tqdm import tqdm
from torch import nn, optim
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#### Загрузка и предобработка необходимых данных

In [43]:
data = pd.read_csv('constraint_train.csv', usecols=['tweet', 'label'])

In [44]:
data.head(3)

,tweet,label
0,The CDC currently reports 99031 deaths. In gen...,real
1,States reported 1121 deaths a small rise from ...,real
2,Politically Correct Woman (Almost) Uses Pandem...,fake


In [45]:
data['label'].value_counts()

label
real    3360
fake    3060
Name: count, dtype: int64

In [47]:
# Разделение данных на признаки и целевую переменную
X = data['tweet']
y = data['label']

In [48]:
# Бинаризируем целевую переменную 
y = y.astype('category').cat.codes

#### Классификация с использованием `sklearn`

In [15]:
# Метод 1: CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

In [49]:
# Векторизация текста
X_cv = count_vectorizer.fit_transform(X)

In [89]:
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_cv, y, test_size=0.2, random_state=42)

In [57]:
# Обучение Decision Tree
model_1 = DecisionTreeClassifier()
model_1.fit(X_train.toarray(), y_train)

DecisionTreeClassifier()

In [77]:
# Предсказание и оценка F1-score
y_pred_tree = model_1.predict(X_test.toarray())
f1_cv_tree = f1_score(y_test, y_pred_tree, pos_label=1)
print(f'CountVectorizer + Decision Tree \nF1-score: {f1_cv_tree}')

CountVectorizer + Decision Tree 
F1-score: 0.896602658788774


Требуемого в задании значения _F1_ достигнуть не удалось, попробуем другую модель

In [90]:
# Обучение Random Forest
model_2 = RandomForestClassifier()
model_2.fit(X_train.toarray(), y_train)

RandomForestClassifier()

In [91]:
# Предсказание и оценка F1-score
y_pred_rf = model_2.predict(X_test.toarray())
f1_cv_rf = f1_score(y_test, y_pred_rf, pos_label=1)
print(f'CountVectorizer + Random Forest Classifier \nF1-score: {f1_cv_rf}')

CountVectorizer + Random Forest Classifier 
F1-score: 0.9168539325842696


Посмотрим на других моделях, как в решении задачи поможет _TfidfVectorizer_

In [79]:
# Метод 2: TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [80]:
# Векторизация текста
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [81]:
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [83]:
# Обучение MultinomialNB()
model_3 = MultinomialNB()
model_3.fit(X_train.toarray(), y_train)

MultinomialNB()

In [84]:
# Предсказание и оценка F1-score
y_pred_mnb = model_3.predict(X_test.toarray())
f1_tfidf_mnb = f1_score(y_test, y_pred_mnb, pos_label=1)
print(f'TfidfVectorizer + MultinomialNB \nF1-score: {f1_tfidf_mnb}')

TfidfVectorizer + MultinomialNB 
F1-score: 0.908445706174592


In [85]:
# Обучение Logistic Regression()
model_4 = LogisticRegression()
model_4.fit(X_train.toarray(), y_train)

LogisticRegression()

In [86]:
# Предсказание и оценка F1-score
y_pred_lr = model_4.predict(X_test.toarray())
f1_tfidf_lr = f1_score(y_test, y_pred_lr, pos_label=1)
print(f'TfidfVectorizer + Logistic Regression \nF1-score: {f1_tfidf_lr}')

TfidfVectorizer + Logistic Regression 
F1-score: 0.9237668161434979


#### Классификация с использованием `pytorch`

In [93]:
# Преобразование данных для PyTorch
X_train_torch = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_torch = torch.tensor(X_test.toarray(), dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.long)
y_test_torch = torch.tensor(y_test.values, dtype=torch.long)

In [94]:
# Нейронная сеть
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train_torch.shape[1], 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [95]:
# Инициализация модели, оптимизатора и функции потерь
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [96]:
# Обучение модели
train_loader = DataLoader(TensorDataset(X_train_torch, y_train_torch), batch_size=32)
for epoch in tqdm(range(10)):
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:28<00:00,  2.87s/it]


In [99]:
# Предсказание и оценка F1-score
model.eval()
with torch.no_grad():
    y_pred_torch = model(X_test_torch)
    y_pred_torch = torch.argmax(y_pred_torch, dim=1)
f1_torch = f1_score(y_test_torch.numpy(), y_pred_torch.numpy(), pos_label=1)
print(f'PyTorch \nF1-score: {f1_torch}')

PyTorch 
F1-score: 0.9277286135693215


#### Итоговые результаты различных моделей

In [115]:
results = pd.DataFrame([['Decision Tree', f1_cv_tree],
                        ['Random Forest', f1_cv_rf],
                        ['MultinomialNB', f1_tfidf_mnb],
                        ['Logistic Regression', f1_tfidf_lr],
                        ['PyTorch', f1_torch]], columns=['Model', 'F1-Score'])
results.sort_values('F1-Score', ascending=False)

,Model,F1-Score
4,PyTorch,0.927729
3,Logistic Regression,0.923767
1,Random Forest,0.916854
2,MultinomialNB,0.908446
0,Decision Tree,0.896603
